## An oblique projection $P$ onto a line

$$
P = \frac{\mathbf{a} \mathbf{b}^T}{\mathbf{a}^T \mathbf{b}}
$$

In [1]:
import numpy as np

def is_symmetric(P, tol=1e-10):
    return np.allclose(P, P.T, atol=tol)

def is_idempotent(P, tol=1e-10):
    return np.allclose(P, P@P, atol=tol)

def is_orthogonal_projection(P, tol=1e-10):
    return is_idempotent(P, tol) and is_symmetric(P, tol)
    
a = np.array([1, 2])
b = np.array([2, 1])
P = np.outer(a, b) / (a @ b)
    
print("\nP")
print(P)
print("\nP is idempotent?:", is_idempotent(P))
print("\nP@P:")
print(P@P)
print("\nP is symmetric?: ", is_symmetric(P))
print("\nP is an orthogonal projection?:", is_orthogonal_projection(P))


P
[[0.5  0.25]
 [1.   0.5 ]]

P is idempotent?: True

P@P:
[[0.5  0.25]
 [1.   0.5 ]]

P is symmetric?:  False

P is an orthogonal projection?: False


## Find max ||Px||


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox

def compute_oblique_projection(a, b):
    P = np.outer(a, b) / (a @ b)
    return P

def is_in_range_P(v, P, tol=1e-8):
    # Check if v is in the range of P by projecting v onto the column space of P
    v_proj = P @ v
    return np.linalg.norm(v - v_proj) < tol

def plot_interactive_projection(a_x, a_y, b_x, b_y, elev, azim, debug_info=False):
    a = np.array([a_x, a_y])
    b = np.array([b_x, b_y])
    P = compute_oblique_projection(a, b)

    # Compute the range, null space, and complementary subspace
    range_P = P @ np.array([1.0, 1.0])  # Resulting shape will be (2,)
    range_PT = P.T @ np.array([1.0, 1.0])  # Resulting shape will be (2,)
    null_P = b * np.array([1, -1])  # Ensure correct 2D form
    range_P_perp = a * np.array([1, -1])  # Ensure correct 2D form

    # Normalize for plotting
    range_P /= np.linalg.norm(range_P)
    range_PT /= np.linalg.norm(range_PT)
    null_P /= np.linalg.norm(null_P)
    range_P_perp /= np.linalg.norm(range_P_perp)

    # Generate unit circle vectors
    theta = np.linspace(0, 2 * np.pi, 10000) 
    x_unit = np.array([np.cos(theta), np.sin(theta)])

    # Apply the projection to each unit vector
    Px = P @ x_unit

    # compute lengths of Px
    lengths = np.sqrt((Px*Px).sum(0)) 

    # Find max||Px||
    eigvals, eigvecs = np.linalg.eig(P.T@P)
    i = eigvals.argmax()
    v = eigvecs[:,i].copy()
    Pv = P @ v
    Pv_length = np.linalg.norm(Pv)

    # Output the result for debugging
    if debug_info:
        print("P:")
        print(P)
        print("\nP is idempotent?:", is_idempotent(P))
        print("\nP is an orthogonal projection?:", is_orthogonal_projection(P))
        print("\nEigenvalues of P.T @ P:")
        print(eigvals)
        print("\nEigenvectors V=[v1 v2] of P.T @ P:")
        print(eigvecs)
        print("\nv:")
        print(eigvecs[:,0])
        print(f"\nv=({v}) maximizes ||Px||")
        print("\n||Pv|| > ||Px|| for all unit vectors x?: ", (Pv_length > lengths).all() )
        print(f"\nv is in Range(P^T)?: {is_in_range_P(v, P.T)}")

    # Plotting in 3D
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot (x1, x2, ||Px||) in 3D
    ax.plot(x_unit[0,:], x_unit[1, :], lengths , label=r'$(x_1, x_2, \|Px\|)$', color='b')

    # scatter plot v and (v[0], v[1], ||Pv||)
    ax.scatter(v[0], v[1], 0, label=r'$v = \text{argmax}\|Px\|_2$', color='lightblue')
    ax.scatter(v[0], v[1], Pv_length, label=r'$\|Pv\|_2 =$' + f'{Pv_length:.4f}', color='c')
    
    # Plot unit circle in grey
    ax.plot(x_unit[0], x_unit[1], 0, label='$x_{unit}$', color='grey')

    # Plot range(P)
    ax.plot([-range_P[0], range_P[0]], [-range_P[1], range_P[1]], [0, 0], color='r', label='Range(P)')

    # Plot range(P^T)
    ax.plot([-range_PT[0], range_PT[0]], [-range_PT[1], range_PT[1]], [0, 0], color='g', label=r'Range($P^T$)')

    # Plot null space and complementary space
    ax.plot([-null_P[0], null_P[0]], [-null_P[1], null_P[1]], [0, 0], color='purple', label='N(P)')
    ax.plot([-range_P_perp[0], range_P_perp[0]], [-range_P_perp[1], range_P_perp[1]], [0, 0], color='orange', label=r'Range$(P)^\perp$')

    # Labels and legend
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel(r'$\theta$')
    ax.set_title(r"Matrix norm $\|P\|_2$ of Oblique projection P")
    ax.legend()
    
    # Set the view angle using the elev and azim values
    ax.view_init(elev=elev, azim=azim)
    
    plt.show()


# Sliders for a_x, a_y, b_x, b_y, elev, and azim
a_x_slider = widgets.FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description='a_x')
a_y_slider = widgets.FloatSlider(value=2.0, min=-5.0, max=5.0, step=0.1, description='a_y')
b_x_slider = widgets.FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description='b_x')
b_y_slider = widgets.FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description='b_y')
elev_slider = widgets.FloatSlider(value=30, min=0, max=90, step=1, description='Elevation')
azim_slider = widgets.FloatSlider(value=120, min=0, max=360, step=1, description='Azimuth')
debug_checkbox = widgets.Checkbox(value=False, description="Debug Info")

# Function to reset the sliders to their default values
def reset_sliders(*args):
    a_x_slider.value = 1.0
    a_y_slider.value = 2.0
    b_x_slider.value = 1.0
    b_y_slider.value = 1.0
    elev_slider.value = 30
    azim_slider.value = 120
    debug_checkbox.value = False
    

# Create a reset button
reset_button = widgets.Button(description="Reset Sliders")

# Link the button to the reset function
reset_button.on_click(reset_sliders)

# Create an interactive plot with the slider values
interactive_plot = widgets.interactive(plot_interactive_projection,
                                       a_x=a_x_slider,
                                       a_y=a_y_slider,
                                       b_x=b_x_slider,
                                       b_y=b_y_slider,
                                       elev=elev_slider,
                                       azim=azim_slider,
                                       debug_info=debug_checkbox)


# Layout and display
ui = VBox([
    interactive_plot.children[-1],
    HBox([a_x_slider, a_y_slider]),
    HBox([b_x_slider, b_y_slider]),
    HBox([elev_slider, azim_slider]),
    HBox([debug_checkbox]),
    reset_button
])

display(ui)